<a href="https://colab.research.google.com/github/achett/IB-Generation/blob/main/IB_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip -q install openai==0.27.0 langchain huggingface_hub
# !pip install --upgrade --quiet  openllm
# !pip install gpt4all
# !pip install langchain_experimental
# !pip install llama-cpp-python

!pip install rouge
!pip install bert-score
!pip install sumy
!pip install langchain
!pip install openai==0.27.0
!pip install python-dotenv
!pip install xmltodict
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.817s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [45]:
import os
import io
import sys
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain, LLMSummarizationCheckerChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
import openai
import textwrap
import dotenv
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
# from langchain_experimental.chat_models import Llama2Chat
from os.path import expanduser
from langchain_community.llms import LlamaCpp
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
import xmltodict
import streamlit as st
import re

In [54]:
######################
# AZURE OPENAI MODEL
######################
dotenv.load_dotenv('/content/drive/MyDrive/Colab Notebooks/IB Generation/.env')

AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "chatgpt" #GPT turbo
TEMPERATURE = float(os.environ.get("TEMPERATURE"))
if 'AZURE_OPENAI_KEY' not in os.environ:
    raise RuntimeError("No 'AZURE_OPENAI_KEY' env var set.  Please see README.")
if 'AZURE_OPENAI_VERSION' not in os.environ:
    raise RuntimeError("No API version specified, please set it up as an environment variable")

# configure azure openai for langchain and/or llm
openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = AZURE_OPENAI_VERSION # this may change in the future
                                    # for langchain, set this version in environment variables using OPENAI_API_VERSION


llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=0, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [ ]:
# ######################
# # LLAMA
# ######################
# model_path = expanduser("/content/drive/MyDrive/Colab Notebooks/IB Generation/models/llama-2-7b-chat.Q4_0.gguf")

# llm = LlamaCpp(
#     model_path=model_path,
#     temperature=0.0,
#     streaming=False,
#     max_tokens=2000
# )
# llm = Llama2Chat(llm=llm)

# template_messages = [
#     SystemMessage(content="Generate a summary of the following text:"),
#     HumanMessagePromptTemplate.from_template("{text}"),
# ]

# prompt_template = ChatPromptTemplate.from_messages(template_messages)


# chain = LLMChain(llm=llm, prompt=prompt_template)

# print(
#     chain.run(
#         text=content
#     )
# )

In [46]:
######################
# DATA
######################
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source.txt', 'r') as file:
    content = file.read()

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target.txt', 'r') as file:
    target = file.read()

content = ' '.join(content.split())
target = ' '.join(target.split())

wrapped_text = textwrap.fill(content,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100,
and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its
toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the
reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period.
Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was
sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and
100 mg/kg groups, no toxicologically significant changes were noted in any examination during the
dosing period. In the 800 mg/kg group, vomiting was observed in males and females on 1 to 5 days
mainly at Weeks 1 and 2 of dosing. Salivation was observed in males and females immediately after
dosing during the dosing period. Decreased body weight was noted in males and females during the
dosing period. Low eryt

In [70]:
######################
# METRICS
######################
from rouge import Rouge
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer

def calculate_rouge2_f1(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_2_f1 = scores[0]['rouge-2']['f']
    return rouge_2_f1

def calculate_bertscore_idf(reference, hypothesis):
    # Using roberta-large model as an example, you can choose other models
    scorer = BERTScorer(model_type='allenai/scibert_scivocab_uncased')
    P, R, F1 = scorer.score([hypothesis], [reference])
    return F1.item()
# roberta-base

In [72]:
########################
# EXTRACTIVE - SUMY: LexRank, Luhn, LSA, TextRank
########################
import re
import nltk
nltk.download('punkt')
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

def count_sentences_and_calculate(text, ratio):
    # Count the number of sentences by splitting based on common punctuation
    sentences = re.split(r'[.!?]+', text)
    # Filter out any empty strings that may result from the split
    sentences = [sentence for sentence in sentences if sentence.strip()]
    sentence_count = len(sentences)

    # Calculate ratio of the sentence count and round to the nearest whole number
    result = round(sentence_count * ratio)

    return result


# Summarizer types
# summarizer = LexRankSummarizer()
# summarizer = LuhnSummarizer()
# summarizer = LsaSummarizer()
summarizer = TextRankSummarizer()

# Set number of sentences to be in summary
summary_ratio = 0.4
sumsent_n = count_sentences_and_calculate(content, summary_ratio)

# Parser
parser = PlaintextParser.from_string(content, Tokenizer("english"))

# Create summary
sum2 = summarizer(parser.document, sumsent_n)

# Combine sentences into one summary
sum_ex = ''
for sentence in sum2:
    sum_ex=sum_ex + str(sentence)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_ex)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_ex)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_ex)/len(content)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-2 F1 Score: 0.49659863446156693
BERTScore F1 (IDF) Score: 0.891338586807251
Length Ratio: 0.5091191307722157


In [148]:
# llm_prompt = ChatPromptTemplate.from_messages([
#         ("system", "Please extract every clause from the following text and reform it as a full sentence:"),
#         ("user", "{prompt}")
#     ])

# output_parser = StrOutputParser()

# chain = llm_prompt | llm | output_parser

# facts = chain.invoke({"prompt":content})

llm_prompt = ChatPromptTemplate.from_messages([
    ("system", "First, carefully read the following text. Extract each sentence. These sentences will be referred to as 'source concepts'."),
    ("user", "{prompt1}"),
    ("system", "Next, process the second text in the same manner to identify 'target concepts'. For each 'source concept', check if an equivalent or highly similar sentence exists among the 'target concepts'. If a match is found, mark it as True; otherwise, mark as False. Please format your response as the total number of True marks over the total number of checks, like '10/20' or '2/5'. Example: If you find 3 sentences from the source concepts match with the target concepts out of 5, respond with '3/5'."),
    ("user", "{prompt2}")
])


output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

facts = chain.invoke({"prompt1":content, "prompt2":sum_abf})

print(facts)

23/23


In [94]:
######################
# ABSTRACTIVE + ZERO SHOT PROMPT
######################
llm_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a summary of the following text:"),
        ("user", "{prompt}")
    ])

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

sum_abz = chain.invoke({"prompt":content})

wrapped = textwrap.fill(sum_abz,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abz)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abz)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abz)/len(content)}")
print(wrapped)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-2 F1 Score: 0.14666666168088904
BERTScore F1 (IDF) Score: 0.885220468044281
Length Ratio: 0.4435389988358556
Project 1 was tested for toxicity in cynomolgus monkeys over a 4-week period with doses ranging from
0 (control) to 800 mg/kg, followed by a 4-week recovery period for the highest dose group. The study
involved equal numbers of male and female monkeys across all dose groups. No deaths or moribundity-
related sacrifices occurred. At doses up to 100 mg/kg, no significant toxicological changes were
observed. However, at 800 mg/kg, adverse effects such as vomiting, salivation, decreased body
weight, alterations in blood parameters (low erythrocyte count, hematocrit value, and hemoglobin
concentration), high triglycerides, high glucose, and changes in liver and adrenal weights were
noted. Toxicokinetic analysis showed that the maximum concentration (Cmax) and area under the curve
(AUC0-24h) of Project 1 increased with dose, with no significant gender differences or marked
diffe

In [100]:
######################
# Abstractive + Few Shot Prompt
######################
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source2.txt', 'r', errors='ignore') as file:
    source2 = file.read()
source2 = ' '.join(source2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source3.txt', 'r') as file:
    source3 = file.read()
source3 = ' '.join(source3.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target2.txt', 'r') as file:
    target2 = file.read()
target2 = ' '.join(target2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target3.txt', 'r') as file:
    target3 = file.read()
target3 = ' '.join(target3.split())

examples = [
    {"input": source2, "output": target2},
    {"input": source3, "output": target3},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

llm_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Generate a summary of the following text:"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

sum_abf = chain.invoke({"input":content})

wrapped = textwrap.fill(sum_abf,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abf)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abf)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abf)/len(content)}")
print(wrapped)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-2 F1 Score: 0.2546583801628024
BERTScore F1 (IDF) Score: 0.9016180038452148
Length Ratio: 0.49786573535118356
Project 1 was administered orally to cynomolgus monkeys at doses of 0 (vehicle), 10, 30, 100, and
800 mg/kg daily for 4 weeks, with a subsequent 4-week recovery period for the 800 mg/kg group to
assess reversibility of observed toxicities. No deaths or moribund sacrifices occurred. At doses up
to 100 mg/kg, no significant toxicological changes were observed. However, at 800 mg/kg, both males
and females exhibited vomiting (mainly during the first two weeks), salivation post-dosing,
decreased body weight, alterations in blood parameters (low erythrocyte count, hematocrit value, and
hemoglobin concentration in one male and one female, high triglycerides in both sexes, and high
glucose in one male), and changes in organ weights (high relative liver weight in males and low
absolute and relative adrenal weights in one female). Toxicokinetic analysis showed Tmax between 0.5
and

In [ ]:
######################
# ABSTRACTIVE + CHAIN (FACTS AND FACT CHECKER)
######################

def clean_output(output):
    # Regular expression to match ANSI escape sequences
    ansi_escape = re.compile(r'\x1b\[([0-9]+)(;[0-9]+)*m')
    # Remove the escape sequences
    cleaned_output = ansi_escape.sub('', output)
    # Optional: Remove other non-printable characters if necessary
    cleaned_output = re.sub(r'[^\x20-\x7E\n]+', '', cleaned_output)
    return cleaned_output

def run_with_captured_output(chain, **kwargs):
    captured_output = io.StringIO()
    current_stdout = sys.stdout
    sys.stdout = captured_output

    result = None
    try:
        if 'query' not in kwargs:
            raise ValueError("Missing required keyword argument: 'query'")
        result = chain.run(**kwargs)
    finally:
        sys.stdout = current_stdout
    verbose_output = captured_output.getvalue()
    captured_output.close()

    # Clean the captured output
    clean_verbose_output = clean_output(verbose_output)

    return result, clean_verbose_output

# Assuming your LLMSummarizationCheckerChain instance and `content` is defined
checker_chain = LLMSummarizationCheckerChain(llm=llm, verbose=True, max_checks=2)

# Now, when calling run_with_captured_output, ensure the content or any other required arguments are correctly passed
# Adjust this to match the expected parameter name ('query' in this case) and provide the necessary content
sum_abch, verbose_output = run_with_captured_output(checker_chain, query=content)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abch)
bertscore_f1_idf = calculate_bertscore_idf(target, sum_abch)

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abch)/len(content)}")
print(sum_abch)



> Entering new LLMSummarizationCheckerChain chain...


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given some text, extract a list of facts from the text.

Format your output as a bulleted list.

Text:
"""
Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. In the 800 mg/kg group, vomiting was

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_summarization_checker/base.py:107: UserWarning: Directly instantiating an LLMSummarizationCheckerChain with an llm is deprecated. Please instantiate with sequential_chain argument or using the from_llm class method.
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert fact checker. You have been hired by a major news organization to fact check a very important story.

Here is a bullet point list of facts:
"""
- Project 1 was administered orally to 3 male and 3 female cynomolgus monkeys per group at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg once daily for 4 weeks to investigate its toxicity.
- An additional 3 males and 3 females were added to the 800 mg/kg group to assess the reversibility of observed toxicity during a subsequent 4-week recovery period.
- The control group received a 0.5 w/v% methylcellulose solution.
- No animal died or was sacrificed due to moribundity during the dosing or recovery period.
- No toxicologically significant changes were observed in the 10, 30, and 100 mg/kg groups during the dosing period.
- In the 800 mg/kg group, vomiting, salivation, and decreased body weight were observed, mainly in the first two weeks.
-

In [ ]:
######################
# AGENT: FACT EXTRACTOR + SUMMARIZER
######################
def clean_output(output):
    # Regular expression to match ANSI escape sequences
    ansi_escape = re.compile(r'\x1b\[([0-9]+)(;[0-9]+)*m')
    cleaned_output = ansi_escape.sub('', output)  # Remove ANSI escape sequences
    cleaned_output = re.sub(r'[^\x20-\x7E\n]+', '', cleaned_output)  # Remove non-printable characters
    return cleaned_output

# Assuming `agent_executor.invoke` prints verbose output to stdout
def run_agent_with_captured_output(agent_executor, **kwargs):
    captured_output = io.StringIO()
    current_stdout = sys.stdout
    sys.stdout = captured_output

    try:
        result = agent_executor.invoke(**kwargs)
    finally:
        sys.stdout = current_stdout
        verbose_output = captured_output.getvalue()
        captured_output.close()

    clean_verbose_output = clean_output(verbose_output)

    return result, clean_verbose_output


template='''Take on the persona of a clinical trial investigator for the pharmaceutical project listed in the text.
I have a list of steps for you to perform a task. You can use the tools for all the steps.

{tools}

1) Extract all the sentences that you think are facts out of this text. Search in PubMed to see if you can validate any of the sentences.

2) Output a numbered list of all the sentences and whether you think they are facts or not.

3) Try to prioritize each of the sentences based on your knowledge as a clinical trial investigator.

4) Try to condense the sentences into fewer sentences and fewer overall words, while retaining all the factual content and all numerical information.

5) Output a second numbered list of all the condensed sentences and which fact from the original numbered list is included in each of the condensed sentences. You can just display the original sentence number.
If any of the facts from the original list are not included please redo steps 4 and 5 until all are included.

6) Take the condensed list and generate a passage that is coherent, readable and reduces redundancies, while retaining all the factual content and all numerical information.

Use the following format:

Content: the text you are extracting key information out of
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
Thought: I now know the final answer
Final Answer: the generated passage

Begin!

Content: {prompt}
Thought:{agent_scratchpad}'''

tools = [PubmedQueryRun()]

llm_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{prompt}")
    ])


agent = create_react_agent(llm, tools, llm_prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_only_outputs=False)

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

inputs = {"prompt": content}
sum_abag, verbose_output = run_agent_with_captured_output(agent_executor, input=inputs)

# Calculate metrics
rouge2_f1_score = calculate_rouge2_f1(target, sum_abag['output'])
# bertscore_f1_idf = calculate_bertscore_idf(target, sum_abag['output'])

print(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
print(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
print(f"Length Ratio: {len(sum_abag['output'])/len(content)}")
print(sum_abag['output'])



> Entering new AgentExecutor chain...
Thought: I need to extract all the sentences that seem to be factual from the text and then validate these facts using PubMed.

Action: Extract all the sentences that seem to be factual from the text.

1) Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. - Fact
2) Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. - Fact
3) Animals in the control group received 0.5 w/v% methylcellulose solution. - Fact
4) No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. - Fact
5) In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. - Fact
6) In th

In [2]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.106.49.121


In [80]:
#################
# STREAMLIT APP
#################

# Write Streamlit app code to a file
%%writefile app.py
import streamlit as st
import os
import re
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain, LLMSummarizationCheckerChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
import openai
import textwrap
import dotenv
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
# from langchain_experimental.chat_models import Llama2Chat
from os.path import expanduser
from langchain_community.llms import LlamaCpp
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
import xmltodict
import textwrap
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from rouge import Rouge
from bert_score import BERTScorer
from transformers import BertTokenizer, BertModel
import nltk
nltk.download('punkt')
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import io
import sys
import textwrap
import pandas as pd


# LLM
dotenv.load_dotenv('/content/drive/MyDrive/Colab Notebooks/IB Generation/.env')

AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "chatgpt" #GPT turbo
TEMPERATURE = float(os.environ.get("TEMPERATURE"))
if 'AZURE_OPENAI_KEY' not in os.environ:
    raise RuntimeError("No 'AZURE_OPENAI_KEY' env var set.  Please see README.")
if 'AZURE_OPENAI_VERSION' not in os.environ:
    raise RuntimeError("No API version specified, please set it up as an environment variable")

# configure azure openai for langchain and/or llm
openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = AZURE_OPENAI_VERSION # this may change in the future
                                    # for langchain, set this version in environment variables using OPENAI_API_VERSION


llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=0, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

# FUNCTIONS
def calculate_rouge2_f1(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_2_f1 = scores[0]['rouge-2']['f']
    return rouge_2_f1

def calculate_bertscore_idf(reference, hypothesis):
    # Using roberta-large model as an example, you can choose other models
    scorer = BERTScorer(model_type='roberta-base')
    P, R, F1 = scorer.score([hypothesis], [reference])
    return F1.item()

def count_sentences_and_calculate(text, ratio):
    sentences = re.split(r'[.!?]+', text)
    sentences = [sentence for sentence in sentences if sentence.strip()]
    sentence_count = len(sentences)
    result = round(sentence_count * ratio)
    return result

def display_results(summary, target, content):
    rouge2_f1_score = round(calculate_rouge2_f1(target, summary), 2)
    bertscore_f1_idf = round(calculate_bertscore_idf(target, summary), 2)
    length_ratio = round(len(summary) / len(content), 2)

    st.write("### Summary")
    st.write(summary)
    st.write("### Metrics")
    st.write(f"ROUGE-2 F1 Score: {rouge2_f1_score}")
    st.write(f"BERTScore F1 (IDF) Score: {bertscore_f1_idf}")
    st.write(f"Length Ratio: {length_ratio}")

    # Return a dictionary containing the metrics
    return rouge2_f1_score, bertscore_f1_idf, length_ratio


def clean_output(output):
    ansi_escape = re.compile(r'\x1b\[([0-9]+)(;[0-9]+)*m')
    cleaned_output = ansi_escape.sub('', output)
    cleaned_output = re.sub(r'[^\x20-\x7E\n]+', '', cleaned_output)
    return cleaned_output

def run_with_captured_output(chain, **kwargs):
    captured_output = io.StringIO()
    current_stdout = sys.stdout
    sys.stdout = captured_output

    result = None
    try:
        if 'query' not in kwargs:
            raise ValueError("Missing required keyword argument: 'query'")
        result = chain.run(**kwargs)
    finally:
        sys.stdout = current_stdout
    verbose_output = captured_output.getvalue()
    captured_output.close()

    # Clean the captured output
    clean_verbose_output = clean_output(verbose_output)

    return result, clean_verbose_output

# Assuming `agent_executor.invoke` prints verbose output to stdout
def run_agent_with_captured_output(agent_executor, **kwargs):
    captured_output = io.StringIO()
    current_stdout = sys.stdout
    sys.stdout = captured_output

    try:
        result = agent_executor.invoke(**kwargs)
    finally:
        sys.stdout = current_stdout
        verbose_output = captured_output.getvalue()
        captured_output.close()

    clean_verbose_output = clean_output(verbose_output)

    return result, clean_verbose_output

def format_metrics(value):
    """Attempt to format the metric value if it is numeric."""
    try:
        # Convert to float and format if possible
        return "{:.2f}".format(float(value))
    except ValueError:
        # Return as is if conversion fails (e.g., value is "NA")
        return value


# DATA
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source.txt', 'r') as file:
    default_content = ' '.join(file.read().split())
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target.txt', 'r') as file:
    default_target = ' '.join(file.read().split())
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source2.txt', 'r', errors='ignore') as file:
    default_source2 = ' '.join(file.read().split())
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source3.txt', 'r') as file:
    default_source3 = ' '.join(file.read().split())
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target2.txt', 'r') as file:
    default_target2 = ' '.join(file.read().split())
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target3.txt', 'r') as file:
    default_target3 = ' '.join(file.read().split())

# PROMPTS
default_system_prompt = "Generate a summary of the following text:"

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/pb_prompt1.txt', 'r', errors='ignore') as file:
    pb_prompt1 = file.read()
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/pb_prompt2.txt', 'r', errors='ignore') as file:
    pb_prompt2 = file.read()
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/pb_prompt3.txt', 'r', errors='ignore') as file:
    pb_prompt3 = file.read()
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/pb_prompt4.txt', 'r', errors='ignore') as file:
    pb_prompt4 = file.read()

default_agent_template='''Take on the persona of a clinical trial investigator for the pharmaceutical project listed in the text.
I have a list of steps for you to perform a task. You can use the tools for all the steps.

{tools}

1) Extract all the sentences that you think are facts out of this text. Search in PubMed to see if you can validate any of the sentences.

2) Output a numbered list of all the sentences and whether you think they are facts or not.

3) Try to prioritize each of the sentences based on your knowledge as a clinical trial investigator.

4) Try to condense the sentences into fewer sentences and fewer overall words, while retaining all the factual content and all numerical information.

5) Output a second numbered list of all the condensed sentences and which fact from the original numbered list is included in each of the condensed sentences. You can just display the original sentence number.
If any of the facts from the original list are not included please redo steps 4 and 5 until all are included.

6) Take the condensed list and generate a passage that is coherent, readable and reduces redundancies, while retaining all the factual content and all numerical information.

Use the following format:

Content: the text you are extracting key information out of
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
Thought: I now know the final answer
Final Answer: the generated passage

Begin!

Content: {prompt}
Thought:{agent_scratchpad}'''

# RESULTS TABLE
# At the beginning of your Streamlit script, after imports
results_summary = {
    "Extractive": {"Summary": "NA", "ROUGE Score": "NA", "BertScore": "NA", "Length Ratio": "NA"},
    "Zero Shot Prompt": {"Summary": "NA", "ROUGE Score": "NA", "BertScore": "NA", "Length Ratio": "NA"},
    "Few Shot Prompt": {"Summary": "NA", "ROUGE Score": "NA", "BertScore": "NA", "Length Ratio": "NA"},
    "Chained Prompts": {"Summary": "NA", "ROUGE Score": "NA", "BertScore": "NA", "Length Ratio": "NA"},
    "Agent": {"Summary": "NA", "ROUGE Score": "NA", "BertScore": "NA", "Length Ratio": "NA"}
}


# GLOBAL SETTINGS
with st.sidebar:
    st.title("Inputs")
    content = st.text_area("Input your text here:", value=default_content, height=100, key="content1")
    target = st.text_area("Input your target summary here:", value=default_target, height=100, key="target1")
    # Widget to adjust LLM temperature in the sidebar
    st.title("Settings")
    temperature = st.number_input("LLM Temperature:", min_value=0.0, max_value=1.0, value=TEMPERATURE, step=0.01, key="temperature")
    st.write("Current LLM Temperature:", temperature)


# Streamlit UI setup with tabs
tab1, tab2, tab3, tab4, tab5, tab_results = st.tabs(["Extractive", "Zero Shot Prompt", "Few Shot Prompt", "Chained Prompts", "Agent", "Results"])


with tab1:
    st.title("Extractive")
    st.caption('TextRank Algorithm - builds a graph of every sentence of the input text, where each text is linked to its most similar sentences, before running the PageRank algorithm to select the most relevant sentences for a summary.')

    summary_ratio = st.number_input("Summary Ratio:", min_value=0.1, max_value=1.0, value=0.4, step=0.05, key="summary_ratio1")

    if st.button("Update Summary", key="update1"):
        sumsent_n = count_sentences_and_calculate(content, summary_ratio)
        parser = PlaintextParser.from_string(content, Tokenizer("english"))
        summarizer = TextRankSummarizer()
        sum2 = summarizer(parser.document, sumsent_n)
        sum_ex = ''.join(str(sentence) for sentence in sum2)
        rouge_ex, bert_ex, ratio_ex = display_results(sum_ex, target, content)

        # Inside each tab, after generating and displaying results
        results_summary["Extractive"] = {
            "Summary": sum_ex,  # or the appropriate variable holding the summary
            "ROUGE Score": rouge_ex,  # Make sure to calculate or retrieve this value
            "BertScore": bert_ex,  # Make sure to calculate or retrieve this value
            "Length Ratio": ratio_ex  # Make sure to calculate or retrieve this value
        }


with tab2:
    st.title("Zero Shot Prompt")
    st.caption('Only a textual prompt')
    system_prompt = st.text_area("System Prompt:", value=default_system_prompt, height=50, key="system_prompt")


    if st.button("Generate LLM Summary", key="update2"):
        llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=temperature, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)
        # Constructing the LLM prompt based on the input
        llm_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("user", content)
        ])

        output_parser = StrOutputParser()

        # Assuming 'llm' is your language model and it's set up to work with this chaining
        chain = llm_prompt | llm | output_parser

        # Invoke the chain with the content as the prompt
        sum_abz = chain.invoke({"prompt":content})

        # Display the results using the existing function
        rouge_abz, bert_abz, ratio_abz = display_results(sum_abz, target, content)

        # Inside each tab, after generating and displaying results
        results_summary["Zero Shot Prompt"] = {
            "Summary": sum_abz,  # or the appropriate variable holding the summary
            "ROUGE Score": rouge_abz,  # Make sure to calculate or retrieve this value
            "BertScore": bert_abz,  # Make sure to calculate or retrieve this value
            "Length Ratio": ratio_abz  # Make sure to calculate or retrieve this value
        }

with tab3:
    st.title("Few Shot Prompt")
    st.caption('Technique where a small number of examples (shots) are given to the model. This method is used to guide the model in generating responses or completing tasks in a specific manner without extensive training or fine-tuning on a large dataset specific to that task.')
    # Allow editing of system prompt
    system_prompt_fs = st.text_area("Edit System Prompt:", value=default_system_prompt, height=50, key="system_prompt_fs")

    # Use expanders for optional editing of example texts
    with st.expander("Edit Example Sources and Targets"):
        source2 = st.text_area("Source 2:", value=default_source2, key="default_source2", height=100)
        source3 = st.text_area("Source 3:", value=default_source3, key="default_source3", height=100)
        target2 = st.text_area("Target 2:", value=default_target2, key="default_target2", height=100)
        target3 = st.text_area("Target 3:", value=default_target3, key="default_target3", height=100)

    examples = [
        {"input": source2, "output": target2},
        {"input": source3, "output": target3},
    ]

    example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )

    llm_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_fs),
            few_shot_prompt,
            ("human", "{input}"),
        ]
    )

    if st.button("Generate Few-Shot Summary", key="update_fs"):
        llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=temperature, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

        output_parser = StrOutputParser()
        chain = llm_prompt | llm | output_parser  # Ensure 'llm' and 'output_parser' are correctly defined/setup
        sum_abf = chain.invoke({"input": content})  # Adjust as necessary for your actual LLM call structure

        rouge_abz, bert_abz, ratio_abz = display_results(sum_abf, target, content)

        # Inside each tab, after generating and displaying results
        results_summary["Few Shot Prompt"] = {
            "Summary": sum_abf,  # or the appropriate variable holding the summary
            "ROUGE Score": rouge_abz,  # Make sure to calculate or retrieve this value
            "BertScore": bert_abz,  # Make sure to calculate or retrieve this value
            "Length Ratio": ratio_abz  # Make sure to calculate or retrieve this value
        }

# Streamlit UI for Abstractive Summarization + Fact Checking
with tab4:
    st.title("Chained Prompts")
    st.caption("""Langchain Chains - sequences or combinations of components and operations that work within GenAI applications.

Used a pre-built chain which was a series of prompts* chained together to self-verify summary.

1) Create Facts Prompt

2) Check Facts Prompt

3) Revise Summary Prompt

4) Are All True Prompt

Temperature parameter influences fact assessment""")

    with st.expander("View Pre-built Prompts:"):
        pb_prompt1 = st.text_area("Create Facts Prompt:", value=pb_prompt1, key="pb_prompt1", height=100)
        pb_prompt2 = st.text_area("Check Facts Prompt:", value=pb_prompt2, key="pb_prompt2", height=100)
        pb_prompt3 = st.text_area("Revise Summary Prompt:", value=pb_prompt3, key="pb_prompt3", height=100)
        pb_prompt4 = st.text_area("Are All True Prompt:", value=pb_prompt4, key="pb_prompt4", height=100)

    if st.button("Generate Summary with Fact Checking", key="generate_abch"):
        llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=temperature, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

        # Setup the checker chain with correct parameters
        checker_chain = LLMSummarizationCheckerChain(llm=llm, verbose=True, max_checks=2)

        # Run the chain with captured output
        sum_abch, verbose_output = run_with_captured_output(checker_chain, query=content)

        # Display the summarized content
        rouge_abch, bert_abch, ratio_abch = display_results(sum_abch, target, content)

        # Display the captured verbose output within an expander for an interactive view
        with st.expander("Click to view verbose output"):
            # Here we use st.text to display the output as if it were printed to console
            st.markdown(f"""
                <style>
                .wrapper {{
                    word-wrap: break-word;
                    white-space: pre-wrap;
                    word-break: break-all;
                }}
                </style>
                <div class="wrapper">
                {verbose_output}
                </div>
                """, unsafe_allow_html=True)

        # Inside each tab, after generating and displaying results
        results_summary["Chained Prompts"] = {
            "Summary": sum_abch,  # or the appropriate variable holding the summary
            "ROUGE Score": rouge_abch,  # Make sure to calculate or retrieve this value
            "BertScore": bert_abch,  # Make sure to calculate or retrieve this value
            "Length Ratio": ratio_abch  # Make sure to calculate or retrieve this value
        }

with tab5:
    st.title("Agent")
    st.caption("""Langchain Agents - intermediaries that understand instructions, process information, and generate responses or carry out actions,

Agents leverage the capabilities of underlying language models and other integrated services.""")
    # Text area for editing the template
    template = st.text_area("Edit Template:", value=default_agent_template, height=300, key="template")

    if st.button("Generate with Agent", key="generate_abag"):
        llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=temperature, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

        tools = [PubmedQueryRun()]

        llm_prompt = ChatPromptTemplate.from_messages([
            ("system", template),
            ("user", "{prompt}")
        ])

        agent = create_react_agent(llm, tools, llm_prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_only_outputs=False)

        # Adjusted invocation with input dictionary
        inputs = {"prompt": content}
        sum_abag, verbose_output = run_agent_with_captured_output(agent_executor, input=inputs)

        # Assuming display_results function is defined to handle result display
        rouge_abag, bert_abag, ratio_abag = display_results(sum_abag['output'], target, content)

        # Display the captured verbose output within an expander for an interactive view
        with st.expander("Click to view verbose output"):
            st.markdown(f"""
                <style>
                .wrapper {{
                    word-wrap: break-word;
                    white-space: pre-wrap;
                    word-break: break-all;
                }}
                </style>
                <div class="wrapper">
                {verbose_output}
                </div>
                """, unsafe_allow_html=True)

        # Inside each tab, after generating and displaying results
        results_summary["Agent"] = {
            "Summary": sum_abag,  # or the appropriate variable holding the summary
            "ROUGE Score": rouge_abag,  # Make sure to calculate or retrieve this value
            "BertScore": bert_abag,  # Make sure to calculate or retrieve this value
            "Length Ratio": ratio_abag  # Make sure to calculate or retrieve this value
        }

with tab_results:
    st.title("Results Overview")

    for method, metrics in results_summary.items():
        with st.container():
            st.subheader(method)

            if metrics["Summary"] != "NA":
                with st.expander("Click to view summary"):
                    st.write(metrics["Summary"])
            else:
                st.write("Summary not generated.")

            # Adjusting metrics display to handle potential string values
            formatted_metrics = {k: format_metrics(v) for k, v in metrics.items() if k != "Summary"}

            # Displaying metrics directly below each summary
            for metric, value in formatted_metrics.items():
                st.text(f"{metric}: {value}")





Overwriting app.py


In [4]:
34.106.49.121

SyntaxError: invalid syntax (<ipython-input-4-b173f8da1ef0>, line 1)

In [81]:
#################
# RUN APP
#################
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 2.16s
your url is: https://shaggy-ravens-turn.loca.lt
^C
